# Tutorial 2: Random Forest Regression

You can also create content with Jupyter Notebooks. This means that you can include
code blocks and their outputs in your book.

### Important before we start
---
Make sure that you save this file before you continue, else you will lose everything. To do so, go to **Bestand/File** and click on **Een kopie opslaan in Drive/Save a Copy on Drive**!

Now, rename the file into Week4_Tutorial1.ipynb. You can do so by clicking on the name in the top of this screen.

<h2>Tutorial Outline<span class="tocSkip"></span></h2>
<hr>
<div class="toc"><ul class="toc-item">
<li><span><a href="#1.-Introducing the packages" data-toc-modified-id="1.-Introducing-the-packages-2">1. Introducing the packages</a></span></li>
<li><span><a href="#2.-" data-toc-modified-id="2.-Dealing-with-missing-values-3">2. Importing the data</a></span></li>
<li><span><a href="#3.-" data-toc-modified-id="2.-Dealing-with-missing-values-3">4. Dealing with missing values</a></span></li>
<li><span><a href="#4.-" data-toc-modified-id="3.-Training-the-random-forest-model-4">5. Training the random forest model </a></span></li>
<li><span><a href="#5.-" data-toc-modified-id="4.-Prediction-with-the-random-forest-model-5">6. Prediction with the random forest model</a></span></li>
<li><span><a href="#6.-" data-toc-modified-id="5.-Evaluate-performance-6">7. Evaluate performance</a></span></li>
<li><span><a href="#7.-" data-toc-modified-id="4.-Parameter-tuning-5">8. Parameter tuning</a></span></li>
<li><span><a href="#8.-" data-toc-modified-id="5.-Feature-importance-6">9. Feature importance</a></span></li></ul></div>

## Learning Objectives
<hr>

- Learn how to estimate a random forest model.
- Deal with missing values in the model.
- Evaluate performance of the random forest model.
- Tuning the hyperparameters.

## 1. Introducing the packages
<hr>

Within this tutorial, we are going to make use of the following packages: 

[**sklearn**](https://scikit-learn.org/stable/index.html) is an open source machine learning library that supports supervised and unsupervised learning. It also provides various tools for model fitting, data preprocessing, model selection, model evaluation, and many other utilities.

[**seaborn**](https://seaborn.pydata.org/index.html) is a Python data visualization library based on matplotlib. It provides a high-level interface for drawing attractive and informative statistical graphics.

[**NumPy**](https://numpy.org/doc/stable/) is a Python library that provides a multidimensional array object, various derived objects, and an assortment of routines for fast operations on arrays.

[**Pandas**](https://pandas.pydata.org/docs/) is an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.

*We will first need to install these packages in the cell below. Uncomment them to make sure we can pip install them*

In [ ]:
!pip install seaborn

Now we will import these packages in the cell below:

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.tree import plot_tree
import seaborn as sns
sns.set_style("whitegrid",{'axes.grid' : True})
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

## 2. Importing the data
<hr>

Let's start again with reading the data. It is the dataset that we produced in the previous [Tutorial](https://elcok.github.io/BigData_AED/week2/tutorial1.html). 

In [ ]:
data1 = pd.read_csv(r"C:\Users\mtn308\Documents\Bigdatacourse\usadataforweek2tut2.csv")

And let's have a look at the data once more:

In [ ]:
data1.head()

In the following line of code we drop duplicates based on the variable SERIAL. Remember that persons from the same household have the same SERIAL number.

In [1]:
data1 = data1.drop_duplicates(subset='SERIAL', keep="first")

NameError: name 'data1' is not defined

QUESTION: Why do we only want to have one person per household in our random forest model? 
ANSWER

## 3. Dealing with missing values
<hr>


In the following lines of code we try to estimate a random forest model, but we run into an error because we have missing values in the dataset. 

We don't estimate the random forest model on the entire dataset, but we split the dataset in a training and testing sample, because we want to train the model on the training dataset and to test model performance on the test dataset.

In [ ]:
training_data, testing_data = train_test_split(data1, test_size=0.3, random_state=25) 

Next we split the training and testing data in two datasets, where x_train contains the explanatory variables and y_train contains the dependent variable. Recall that COSTENERGY is the sum of COSTELEC, COSTGAS and COSTFUEL, so they should also be removed from the explanatory variables.  

In [ ]:
x_train = training_data.drop(['COSTENERGY', 'COSTELEC', 'COSTGAS', 'COSTFUEL'], axis = 1)
y_train = training_data[['COSTENERGY']]
y_train = y_train.to_numpy() #here we convert y_train from a pandas dataframe with one column to a numpy array.   

x_test = testing_data.drop(['COSTENERGY', 'COSTELEC', 'COSTGAS', 'COSTFUEL'], axis = 1)
y_test = testing_data[['COSTENERGY']]
y_test = y_test.to_numpy()

Training and fitting a random forest model is done in one line of code:

In [ ]:
rf = RandomForestRegressor(n_estimators = 50, max_features = 'sqrt', max_depth = 5, random_state = 18).fit(x_train, y_train)

QUESTION: Explain in your own words the options in the RandomForestRegressor and how these options will affect the results. Also explain two other options in RandomForestRegressor that are not included in the line of code above. 

The random forest model didn't run because there are missing values in the data. There are several options to deal with missing data. We could for example replace all the missing values by 0 or by the mean of that variable. First, let's check again which variables have missing values. 

In [ ]:
data1.isnull().sum().sort_values(ascending = False)

There are quite some missing values in the dependent variable COSTENERGY. And the number of NA's in COSTENERGY is suspiciously similar to the number of missing values in some other variables, such as BUILTYR, ROOMS, HHINCOME, etc. Let's check what happens if we remove the rows from the dataset when there is no value for COSTENERGY. Probably we can remove a lot of missing values all at once.  

In this line of code we remove all rows (axis = 0 indicates rows, and axis = 1 indicates columns) for which COSTENERGY is NA.

In [ ]:
datanan = data1.dropna(axis=0, subset='COSTENERGY') 

In [ ]:
## Then check if this has removed a lot of NA's in other variables as well. 
datanan.isnull().sum().sort_values(ascending = False)

Indeed for 15145 households there was a lot of data missing and we can savely remove these households from the dataset, because 1) these instances are not very meaningful and 2) we could perfectly predict them if we keep them in the dataset using the missing values of VEHICLES, ROOMS, HHINCOME, etc.

Then we have a handful variables left with missing values. Explain what you would do with these missing values? There is not necessarily a right or wrong answer, but ofcourse your choices matter for the results. 
ANSWER

In [ ]:
## Replace the missing values in all variables with code. You can check the code of the previous tutorial on how to do this. 
# ANSWER
datanan['RENT'] = datanan['RENT'].fillna(0) 
datanan['DEGFIELD'] = datanan['DEGFIELD'].replace(np.nan, 0) 
datanan['MORTGAG2'] = datanan['MORTGAG2'].replace(np.nan, 0) 

datanan['MORTGAGE'] = datanan['MORTGAGE'].replace(np.nan, 0) 
datanan['PROPINSR'] = datanan['PROPINSR'].replace(np.nan, 0) 
datanan['IND'] = datanan['IND'].replace(np.nan, 0) 
datanan['OCC'] = datanan['OCC'].replace(np.nan, 0) 
datanan['EMPSTAT'] = datanan['EMPSTAT'].replace(np.nan, 0) 
datanan['INCTOT'] = datanan['INCTOT'].replace(np.nan, 0) 
datanan['POVERTY'] = datanan['POVERTY'].replace(np.nan, 0) 

## 4. Training the random forest model
<hr>

Now we can actually estimate the random forest model. Note that we use training and estimation of the random forest model interchangebly. In machine learning, we usually talk about training a model, while in statistics/econometrics, we would say estimating a model, but both words mean essentially the same.

In [ ]:
# Remember to split the data again in a training and testing set. Now we use the datanan dataset which has no missing values.   
training_data, testing_data = train_test_split(datanan, test_size=0.3, random_state=25) 

x_train = training_data.drop(['COSTENERGY', 'COSTELEC', 'COSTGAS', 'COSTFUEL'], axis = 1)
y_train = training_data[['COSTENERGY']]
y_train = y_train.to_numpy()

x_test = testing_data.drop(['COSTENERGY', 'COSTELEC', 'COSTGAS', 'COSTFUEL'], axis = 1)
y_test = testing_data[['COSTENERGY']]
y_test = y_test.to_numpy()

In [ ]:
rf = RandomForestRegressor(n_estimators = 100, max_features = 'sqrt', max_depth = 3, random_state = 18).fit(x_train, y_train)

To get some insight in the random forest model, we can visualize the first tree of the model. [0] calls the first tree, but we can also plot any other tree of the model.  

In [ ]:
firsttree = rf.estimators_[0]
plt.figure(figsize=(22,8))
plot_tree(firsttree, feature_names=x_train.columns, filled=True, fontsize = 11)
plt.show()

QUESTION: Interpret what you see in the decision tree. Hint: Explain the terms squared_error, samples and value in the tree. 

## 5. Prediction with the random forest model
<hr>

In the next line of code, we use the trained random forest model to predict the dependent variable of the testing dataset. We use the function predict of the random forest regressor.  

In [ ]:
y_pred = rf.predict(x_test)

## 6. Evaluate performance
<hr>

Using the predicted dependent variable, we can evaluate the performance of the model. We first do this visually by plotting the dependent variable of the testing dataset together with the predicted values.

In [ ]:
fst = 12
plt.plot(y_test,'bo')
plt.plot(y_pred,'ro')
plt.legend(['Observed','Predicted'], fontsize = fst)
plt.xticks(fontsize=fst)
plt.yticks(fontsize=fst)
plt.ylabel('ENERGYCOST',fontsize=fst)

QUESTION: Interpret the plot.

Next we check performance in terms of a numeric criterion. We can for example check the mean squared error or the R2. 

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print(mse)

In [ ]:
r2 = r2_score(y_test, y_pred)
print(r2)

In [ ]:
## Difference r2 and mean squared error? 

## 7. Parameter tuning
<hr>

The R2 score is pretty low, so we want to try other hyperparameters. We will change the depth and the number of trees in the forest. In the code below, we run a for loop that tries 3 different n_estimators and 3 different depths, so we estimate 9 different random forest models. We save the mse and r2 of each model in a list, i.e. list_mse and list_r2. 

In [ ]:
list_mse = []
list_r2 = []

for n_est in [20,50,100]:
    for dep in [4,5,6]:
       rf = RandomForestRegressor(n_estimators = n_est, max_features = 'sqrt', max_depth = dep, random_state = 18).fit(x_train, y_train) 
       y_pred = rf.predict(x_test)
       mse = mean_squared_error(y_test, y_pred)
       r2 = r2_score(y_test, y_pred)
       list_mse.append(mse)
       list_r2.append(r2)
        
print(list_mse)
print(list_r2)

QUESTION: Interpret the results. Which model performs best (think about how the for loop works, or use print(n_est) and print(dep) in the for loop to make life easier). 

QUESTION: Do you want to try another parameter space? Also think about the time the model needs to run. 
The higher the number of trees and the depth, the longer it takes to train the model.
If yes, you can copy the code from the previous cell and change the numbers over which the for loop iterates, i.e. the numbers in [20,50,100] and [4,5,6].  

In [ ]:
## ANSWER. 

## 8. Feature importance
<hr>

Next we want to see which variables have the most predictive value for ENERGYCOSTS. We can check the importance of each feature in the random forest model by calling rf.feature_importances_. We make a horizontal barplot to see the importance of each feature. You can fill in the hyperparameters.

In [ ]:
rf = RandomForestRegressor(n_estimators = 50, max_features = 'sqrt', max_depth = 10, random_state = 18).fit(x_train, y_train) 
sorted_idx = rf.feature_importances_.argsort() #we sort the importance of features from high to low. 
plt.figure(figsize=(7, 10.5))
plt.barh(x_train.columns[sorted_idx], rf.feature_importances_[sorted_idx])
plt.show()

QUESTION: According to the plot, which variables, also called features, are most important in the random forest? Should we leave out variables from the model. Hint: Not all variables in the model are meaningful, are they? 